In [ ]:
import sys
sys.path.insert(0, '../')

import uuid
import os
import re
import json 
import time
import random
from typing import Any, Callable, Generator

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.prompts.base import BaseOutputParser
from langchain.agents import load_tools, Tool
from langchain.agents import initialize_agent
from langchain.llms import OpenAI

from chat.base import BaseChat, ChatHistory, Response
from chat.widget_search import ChatOutputParser, WIDGET_INSTRUCTION, SEARCH_INSTRUCTION, TEMPLATE, IDENTIFY_TEMPLATE
from chat.base import *
from index.weaviate import *
from index.widgets import *
from utils import *
from config import *
from system import *
from tools import *

set_api_key()

def send_message(resp, before_message_id=None, last_chat_message_id=None):
    if resp.operation == 'replace':
        print(resp.response)
    return 1

system = initialize(default_config)

In [ ]:
s = '\\'
with open(f"{'/'.join(os.getcwd().split(s)[:-1])}/knowledge_base/widgets.txt", 'r') as f: widgets = f.read()
# widgets = re.sub(re.compile(r"<\||\|>|{|}"), '', widgets).replace('-', '_').replace('Widget magic command', 'Function').replace('widget', 'function')
widgets_k = '---'.join(random.choices(widgets.split('---'), k=15))
system_message = f"""You have to imitate a human user who asks a chatbot queries related to web3. The queries can be short instructions to do a task.
The queries should be based on the below widget commands as the bot invokes one of them to respond. 
When bot returns "response DONE", consider it answered your query. You can start a new conversation after it. Don't start a new conversation if the previous hasn't ended.
Ask one question at a time and continue to chat with the bot as a user.

{widgets_k}"""

messages = [SystemMessage(content=system_message)]
history = ChatHistory([], uuid.UUID, '0x4eD15A17A9CDF3cc7D6E829428267CaD67d95F8F')
llm = ChatOpenAI(model_name="gpt-4", max_tokens=60, temperature=0.5,)

In [ ]:
before_message_id, message_id = None, uuid.UUID
while True:
    user_message = llm(messages)
    messages.append(user_message) # AIMessage
    print(f"User Input: {user_message.content}")
    print('----')

    system.chat.receive_input(history, user_message.content, send_message, message_id, before_message_id)
    messages.append(HumanMessage(content=history.messages[-2].content))
    print(f"Cacti Bot: {history.messages[-2].content}")

    if "first_widget_command" in history.messages[-1].content:
        messages.append(HumanMessage(content="response DONE")) 
        print(f"Cacti Bot: response DONE")
    else:
        user_message = llm(messages)
        if user_message.content=="response DONE":
            messages.append(HumanMessage(content="response DONE")) 
            print(f"Cacti Bot: response DONE ---")
    print()